# Jupiter Notebook to test csv_stuff.py

In [2]:
from operator import ne
import pandas as pd
import sys
import re
import random
from bs4 import BeautifulSoup
import argparse
import matplotlib.pyplot as plt

# df_labled = []


def create_labled_table_routine(semester, ha, tasks, prog_language, labled_csv=None):
    csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    keep_columns = ['Nachname', 'Vorname'] + tasks
    drop_columns = []
    for x in df.columns:
        if x not in keep_columns:
            drop_columns.append(x)
    df = df.drop(drop_columns, axis=1)
    df = add_valid_code_columns(df=df, semester=semester, ha=ha, tasks=tasks, prog_language=prog_language)
    df_labled = create_labled_table(df, semester, ha, tasks, prog_language)
    return df_labled, len(df_labled.index)


def create_labled_table(df, semester, ha, tasks, prog_language):
    df_labled = pd.DataFrame(columns=['id', 'semester', 'ha', 'task', 'prog_lang', 'surname1', 'lastname1', 'surname2', 'lastname2', 'code1', 'code2', 'label'])
    i = 0
    for task in tasks:
        for ln1, sn1, code1 in df.loc[df[f'{task} empty']==0][['Nachname', 'Vorname', task]].values:
            for ln2, sn2, code2 in df.loc[(df[f'{task} empty']==0) & (df[task] != code1)][['Nachname', 'Vorname', task]].values:
                df_labled.loc[len(df_labled)] = [i, semester, ha, task, prog_language, sn1, ln1, sn2, ln2, code1, code2, -1]
                i += 1
    return df_labled



"""
(empty- or template solution) == 1
"""
def add_valid_code_columns(df, semester, ha, tasks, prog_language):
    for task in tasks:
        answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
        answerpreload = answerpreload.replace('\t', '').replace('\r', '').replace('\n', '')
        #every special character beetween \Q and \E is ignored, but the inserted \S*; answers shouldnt contain \Q and \E!
        answerpreload = re.escape(answerpreload)
        #{{ cr_random.f1 }} --> \S*
        answerpreload = re.sub(r"\\{\\{\\\s*\S+\s*\\}\\}", r"\\S*", answerpreload)
        summe=0
        result = 0
        df[f"{task} empty"] = 0
        #TODO wie vergleicht man leicht unterschiedliche texte auf Gleichheit miteinander??? -> bisher nur direkte gleichheit
        for j,column in enumerate(df[task]):
            #there are \\n in the student solutions because "...\n" --> "...\\n" while read_csv
            column = column.replace('\t', '').replace('\r', '').replace('\n', '').replace('\\n', r'\n')
            # test if template was given or not
            if answerpreload != '':
                result = re.match(answerpreload, column)
            if column == '-' or result:
                df.loc[j, f"{task} empty"] = 1
                summe+=1
        print(summe)
    return df


def get_given_code(file):
    with open(file) as xmlstr:
        soup = BeautifulSoup(xmlstr, 'xml')
        answerpreload = soup.find('answerpreload').text
        return answerpreload


def get_new_pair(df_labled):
    next = df_labled.loc[df_labled['label']==1,['surname1', 'surname2', 'code1', 'code2']]
    if not next.empty:
        next = next.values[0]
        return next[0], next[1], next[2], next[3]
    print('no unlabled pair')
    return None



#TODO 1. erfolgreichen match hinbekommen        *check*
#TODO 2. empty_solution_matrix richtig setzen   *check*
#TODO 3. dann daraus ein paar ableiten          *check*
#TODO 4. dann daraus eine tabellenreihe für ...labled.csv machen *check*
#TODO 5. dann eine ganze tabelle draus machen   *check*
#TODO 6. wo wird tabelle zwischengespeichert?
#TODO 7.1 funktion schreiben die die erste Routine macht mit #aller paar und #aller bereits gelableten und die 3 texte ausgibt
#TODO 7. funktion schreiben, die ein lable übergeben bekommt, in tabelle einträgt und ein neues paar zurückgibt (ggf. auch neue vorgabe)
#TODO gibt es ein Download fenster?
#TODO checken weshalb nur 12/13 leere abgaben bei antwort 10 gefunden wurden
#TODO beim überspringen erstmal mit -1 lablen?; double linked list, für die id schreiben, um prev und next button zu realisieren


In [3]:
semester='SoSe21'
ha='9'
# tasks = ['Antwort 9']
tasks = ['Antwort 8', 'Antwort 9', 'Antwort 10']
prog_language='C'
df_labled, count = create_labled_table_routine(semester=semester, ha=ha, tasks=tasks, prog_language=prog_language)
# answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{tasks[0]}.xml')

4
13
12


In [25]:
# print(df_labled.loc[0:5])
last_id=4
label_score=0.76
# try:
already_labled = df_labled.loc[last_id]['label']
print(already_labled)
df_labled.loc[last_id]['label'] = label_score
# print(df_labled)
print(df_labled.loc[(df_labled['label']==-1) & (df_labled['id'] > last_id)])
# print(df_labled.loc[len(df_labled.index)-1, 'label'])
# except:
#     print('diesen index gibt es nicht')


-1
        id semester ha        task prog_lang           surname1 lastname1  \
5        5   SoSe21  9   Antwort 8         C         Yang Felix      Wang   
6        6   SoSe21  9   Antwort 8         C         Yang Felix      Wang   
7        7   SoSe21  9   Antwort 8         C         Yang Felix      Wang   
8        8   SoSe21  9   Antwort 8         C         Yang Felix      Wang   
9        9   SoSe21  9   Antwort 8         C         Yang Felix      Wang   
...    ...      ... ..         ...       ...                ...       ...   
1345  1345   SoSe21  9  Antwort 10         C  Karsten Siegfried    Hansel   
1346  1346   SoSe21  9  Antwort 10         C  Karsten Siegfried    Hansel   
1347  1347   SoSe21  9  Antwort 10         C  Karsten Siegfried    Hansel   
1348  1348   SoSe21  9  Antwort 10         C  Karsten Siegfried    Hansel   
1349  1349   SoSe21  9  Antwort 10         C  Karsten Siegfried    Hansel   

                   surname2       lastname2  \
5               Max Richa

In [46]:
semester='SoSe21'
ha=['6','7','8','9','10']   #5 relevant programm homework weeks + (possible exam tasks)
abgaben=[34,34,34,34,34]    #SoSe2021 34 student only in c
tasks = [4, 3, 3, 3, 3]     #2-4 programm tasks per week
summe=0
for i,j in zip(abgaben,tasks):
    summe += (i-1)**2*j     #with 34 comes 33 pairs per student per task per week
print(summe)                #number of homework pairs in one semester
print(summe*4)              #number of pairs in four semester
print(summe*4*2)            #number of pairs in four semester in 2 programming languages


17424
69696
139392
